# Simple Stock Price Prediction (GME)

In [1]:
import yfinance as yf
import pandas as pd

In [7]:
gme = yf.Ticker('GME')
gme = gme.history(period='max')

gme.to_csv('data/gme.csv')

In [8]:
gme

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-02-13,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
2002-02-14,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2002-02-15,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
2002-02-19,1.666417,1.666417,1.578047,1.607504,7410400,0.0,0.0
2002-02-20,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0
...,...,...,...,...,...,...,...
2022-07-25,35.000000,35.509998,33.250000,33.980000,5686300,0.0,0.0
2022-07-26,32.869999,33.310001,32.099998,32.430000,4866100,0.0,0.0
2022-07-27,32.959999,34.119999,32.160000,33.779999,4251200,0.0,0.0


In [3]:
del gme['Dividends']
del gme['Stock Splits']
gme

TypeError: 'NoneType' object does not support item deletion

In [ ]:
gme['Close'].plot()

In [ ]:
import pandas as pd

output_var = pd.DataFrame(gme['Close'])

features = ['Open', 'High', 'Low', 'Volume']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(gme[features])
feature_transform = pd.DataFrame(columns=features, data=feature_transform, index=gme.index)
feature_transform

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

timesplit = TimeSeriesSplit(n_splits=10)

for train_index, test_index in timesplit.split(feature_transform):
    x_train, x_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
    y_train, y_test = output_var[:len(train_index)].values.ravel(), output_var[len(train_index): (len(train_index)+len(test_index))].values.ravel()

In [ ]:
import numpy as np

train_x = np.array(x_train)
test_x = np.array(x_test)

x_train = train_x.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = test_x.reshape(x_test.shape[0], 1, x_test.shape[1])

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(1, train_x.shape[1]), activation='relu', return_sequences=False))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=75, batch_size=8, verbose=1, shuffle=False)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_test, label='True Value')
plt.plot(y_pred, label='LSTM Value')
plt.title('Prediction by LSTM Value')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.show()